# Inserção de elementos
Notebook dedicado a incrementar as funções que serão usadas para adicionar

#### Importando bibliotecas e importando as funções primárias

In [2]:
#importa as bibliotecas
import pandas as pd
import xlrd
import pickle #modulo usado para serializar dados para o arquivo binario
import os
import time
import copy
from unicodedata import normalize #biblioteca usada para criar as arvores TRIE

In [3]:
#Funcao que retira as features de artistas selecionados e salva apenas o artista principal da musica
def retiraFeaturing(artistaString):
    
    if " Featuring" in artistaString:   # Se a música tiver um featuring, deve-se retirá-lo
            indexFeat = artistaString.find(" Featuring")   # pega índice do char space antes de Featuring
            artistaString = artistaString[:indexFeat]   # retorna apenas o primeiro artista
        
    return artistaString   # caso não caia no if, simplesmente retorna o artista assim como veio

# Função que formata string desconsiderando maiúsculas e minúsculas e que remove acentos
def formataString(string):
    string = normalize('NFKD', string).encode('ASCII', 'ignore').decode('ASCII')
    return string.upper()

#------------------------------------------------------------
#Definicao das funcoes usadas para gerar as arvores TRIE
#------------------------------------------------------------
#Definicao da classe de nodo das arvores TRIE
class TrieNode(object):
    def __init__(self, char: str):   # __init__ é um método especial para fazer construtores
        self.char = char   # caractere do nodo atual
        self.filhos = []   # nodos filhos
        self.pFinalizada = False   # se é o último nodo e a palavra terminou
        self.indices = []   # lista vazia para nodos que não são término de palavra
        ### não é um índice único e sim uma lista, pois podem existir títulos de música repetidos com mais de um artista

#Definicao da funcao que adiciona uma string na arvore TRIE
def addString(raiz, palavra: str, indice: int):   # função usada para adicionar uma palavra nova à estrutura trie 
    palavra = formataString(palavra)   # formata string de entrada para o padrão (todas as letras maiúsculas sem acento)
    nodo = raiz
    for char in palavra:
        encontradoEmFilho = False
        # busca pelo caractere nos filhos do nodo atual
        for filho in nodo.filhos:
            if filho.char == char: 
                nodo = filho   # apontamos o nodo para o filho que contém esse char
                encontradoEmFilho = True
                break
        
        if not encontradoEmFilho:   # se o caractere não foi encontrado, adiciona novo filho
            novoNodo = TrieNode(char)
            nodo.filhos.append(novoNodo)
            nodo = novoNodo   # apontamos, então, o nodo para seu novo filho e continuamos a iteração
    
    nodo.pFinalizada = True   # indica que até ali pode ser uma palavra (nome de música/artista completo)
    nodo.indices.append(indice)   # assinala o indice passado
    
#Funcao que procura uma string na arvore TRIE 
def findString(select, dados, raiz, palavra: str) -> (bool, []):
    """
      1. Se a palavra existe e tem algum índice associado, retorna verdadeiro e a lista de índices dela
      2. Se a palavra não existe, retorna falso e uma lista com sugestões de palavras
      
      Sendo o parâmetro select utilizada para indicar se a busca é por artista ou música (0 música, 1 artista)
    """
    palavra = formataString(palavra)   # formata string de entrada para o padrão (todas as letras maiúsculas sem acento)
    nodo = raiz
    stringSugerida = ""  # guarda os caracteres já encontrados para sugerir os próximos
    
    if not raiz.filhos:   # se o nodo raiz não tiver nenhum filho, trivial, retorna falso
        return False, []
    for char in palavra:
        charNaoEncontrado = True
        for filho in nodo.filhos:
            if filho.char == char:
                charNaoEncontrado = False   # assinala que o char foi encontrado
                stringSugerida = stringSugerida+char
                nodo = filho   # passa iteração para o nodo filho
                break
        
        if charNaoEncontrado:   # chama a função que procura sugestões e retorna a tupla (False, lista de sugestões)
            return False, sugereStrings(select, dados, nodo, stringSugerida, 15)
    
    # Caso passe por todos os caracteres sem retornar false, então significa que a palavra foi encontrada
    # Resta saber se aquele nodo é um nodo final com um índice associado
    if nodo.pFinalizada:
        return True, nodo.indices
    else:
        return False, sugereStrings(select, dados, nodo, stringSugerida, 15)
    
#Funcoes utilizadas para sugerir buscas
def sugereStrings(select, dados, nodo, string, maxSugestoes):   # onde select igual a 1 significa que se procura um artista e False, música
        listaDeSugestoes = []   # cria lista vazia para adicionarmos as sugestões
        procuraSugestoes(nodo, string, listaDeSugestoes)   # chama a função que percorre a trie procurando sugestões
        
        
        itera = copy.deepcopy(listaDeSugestoes)   # utilizado para iteração na lista que está sendo modificada enquanto o loop itera
        
        # substitui os indices pelos títulos
        for indice in itera:
            listaDeSugestoes.remove(indice)   # remove o índice 
            if select:
                listaDeSugestoes.append(dados[indice]['Artista']) 
            else:
                listaDeSugestoes.append(dados[indice]['Titulo']) 
            
        
        listaDeSugestoes = sorted(listaDeSugestoes)   # ordena a lista alfabeticamente
        return listaDeSugestoes[:maxSugestoes]   # retorna apenas os n elementos da lista definidos por maxSugestoes

    
# função que dado um nodo, uma parte correta de uma string (que existe na TRIE), uma lista de sugestoes, retorna uma lista de sugestões
def procuraSugestoes(nodo, string, listaDeSugestoes):
        
        if nodo.pFinalizada:   
            listaDeSugestoes.append(nodo.indices[0])   # adiciona nova sugestão na lista
        
        for filho in nodo.filhos:
            
            procuraSugestoes(filho, string+filho.char, listaDeSugestoes)   # recursão para cada filho 
            if filho.pFinalizada:   
                listaDeSugestoes.append(filho.indices[0])   # adiciona nova sugestão na lista
                
        return  

### Função de inserção de anos
Aceita como entrada o nome de um arquivo .xlsx, implementado assim para adicionar blocos de anos conforme a billboard liberar mais data

In [11]:
def adicionaAno(arquivo):
    
    df = pd.read_excel(arquivo) #Le o arquivo de excel para a variavel Dataframe
    
    #----------------------------------------------------------
    #Abrindo os arquivos com os tops de artistas
    with (open('topArtistas.bin', 'rb')) as openfile:
        topArtistas = pickle.load(openfile)
        
    with (open('topMusicas.bin', 'rb')) as openfile:
        topMusicas = pickle.load(openfile)
        
    #Abrindo os arquivos com as arvores Trie      
    with (open('trieMusicas.bin', 'rb')) as openfile:
        trieMusicas = pickle.load(openfile)
        
    with (open('trieArtistas.bin', 'rb')) as openfile:
        trieArtistas = pickle.load(openfile)
    #----------------------------------------------------------
    contador = {}
    #Salva os registros do ano atual
    for i in range(len(df)):
        ano = df['WeekID'][0].split('-')
        ano = ano[0]
        if(df['SongID'][i] in contador): #testa se a musica ja esta no dicionario (ja foi computada)
            contador[df['SongID'][i]]['Pontos'] += (101 - df['Week Position'][i])
        else: #cai aqui se ainda nao foi computada
            contador[df['SongID'][i]] = {}
            contador[df['SongID'][i]]['Artista'] = df['Performer'][i]
            contador[df['SongID'][i]]['Titulo'] = df['Song'][i]
            contador[df['SongID'][i]]['Ano'] = ano
            contador[df['SongID'][i]]['Pontos'] = (101 - df['Week Position'][i])
            #A medida 'Peak' nao eh a posicao exata que ela atingiu mas os "pontos maximos" ganhos em uma semana (de 1 a 100)
            contador[df['SongID'][i]]['Peak'] = (101 - df['Peak Position'][i]) #salva qual foi a maior posicao alcançada pela musica (usada para criterios de desempate)
            contador[df['SongID'][i]]['Semanas'] = df['Weeks on Chart'][i] #Terceiro criterio de desempate
        
    #Faz o sort do ano atual
    items = sorted(contador.items(), key = lambda tup: (tup[1]["Pontos"], tup[1]["Peak"], tup[1]["Semanas"]), reverse=True)
    
    #armazena as primeiras 200 entradas em um dicionário mais organizado
    top = [dict() for x in range(200)] #cria uma lista de 200 dicionarios
    for i in range(len(top)):
    
        for y in items[i][1]:
            top[i][y] = items[i][1][y]
        
        # Adiciona no dicionário de todos os artistas (ou soma os pontos) para cianção do arquivo topArtistas.bin
        artistaIterado = retiraFeaturing(str(top[i]['Artista']))   # retira o featuring caso tenha
        
        indArtista = -1 #flag que salva o indice do artista iterado, se for -1 o artista nao esta na lista
        indice = 0 #indice que esta procurando o artista
        for x in topArtistas: #percorre o arquivo topArtistas procurando pelo artista
            if x[0] == artistaIterado:
                indArtista = indice
            indice += 1

        if(indArtista != -1): #Se o artista ja esta na lista
            topArtistas[indArtista] = (artistaIterado, topArtistas[indArtista][1] + top[i]['Pontos'])
        else: #cria um novo artista
            topArtistas.append((artistaIterado, top[i]['Pontos'])) #da um append com a tupla 'nome artista' + 'pontos'
    
    #Concatena o dicionario no arquivo teste.bin
    filename = 'database.bin'
    topBytes = []
    

    if os.path.exists(filename): #confere se o arquivo existe
        with open(filename,'rb') as openfile:  #with automaticamente da um close() no final
            topBytes = pickle.load(openfile) #carrega o que ja esta salvo no arquivo

    indIni = len(topBytes) #Salva o primeiro bit dos dados novos que foram inseridos  
    
    newData = top #Aqui vem a nova data que deve ser concatenada
        
    topBytes = topBytes + newData #concatena o que ja tinha no arquivo binario com a nova data

    with (open(filename, 'wb+')) as openfile: #abre o arquivo no modo de leitura binaria 'wb' como openfile
        pickle.dump(topBytes, openfile) 
        
    #-----------------------------------------------------------------------------    
    #Cria o arquivo de indice para consultas    
    #-----------------------------------------------------------------------------
    filename = 'indices.bin'
    fileBytes = []

    if os.path.exists(filename): #confere se o arquivo existe
        with open(filename,'rb') as openfile:  #with automaticamente da um close() no final
            fileBytes = pickle.load(openfile)


    newData = {'Ano': ano, 'Min': len(topBytes) - 200, 'Max': len(topBytes) - 1} #Aqui vem a nova data que deve ser concatenada
        
    fileBytes.append(newData)#concatena o que ja tinha no arquivo binario com a nova data
    with (open(filename, 'wb+')) as openfile: #abre o arquivo de teste no modo de leitura binaria 'wb' como openfile
        pickle.dump(fileBytes, openfile)
     
    
    #-----------------------------------------------------------------------------
    #Cria e ordena o topMusicas - Reaproveitada da criação de arquivos
    #-----------------------------------------------------------------------------
    itera = len(topMusicas)
    for x in top:
        topMusicas.append((itera, x['Pontos']))
        if(itera < len(topMusicas)-1): #Vai incrementando ate chegar no final da lista
            itera += 1    

    topMusicas.sort(key=lambda tup: tup[1], reverse=True)  #Faz um sort usando o elemento [1] (pontos) da tupla como criterio de ordenacao
    
    #Salva ranking geral de musicas
    with (open("topMusicas.bin", 'wb+')) as openfile:
        pickle.dump(topMusicas, openfile)  
        
    topArtistas.sort(key=lambda tup: tup[1], reverse=True)  #Faz um sort usando o elemento [1] (pontos) da tupla como criterio de ordenacao
    
    #Salva ranking geral de artistas
    with (open("topArtistas.bin", 'wb+')) as openfile:
        pickle.dump(topArtistas, openfile)
        
    #-----------------------------------------------------------------------------    
    #Inserindo nas arvores TRIE
    #-----------------------------------------------------------------------------
    for x in range(indIni, len(topBytes)):
        addString(trieMusicas, str(topBytes[x]['Titulo']), x)
        addString(trieArtistas, topBytes[x]['Artista'], x)
        
    filename = "trieMusicas.bin"
    with (open(filename, 'wb+')) as openfile: #abre o arquivo de teste no modo de leitura binaria 'wb' como openfile
        pickle.dump(trieMusicas, openfile)
    
    filename = "trieArtistas.bin"
    with (open(filename, 'wb+')) as openfile: #abre o arquivo de teste no modo de leitura binaria 'wb' como openfile
        pickle.dump(trieArtistas, openfile)

### Função de inserção de registros individuais

In [101]:
#aceita como entrada strings nos atributos de artista e titulos e ints nos de ano,pontos,peak e semanas
def adicionaReg(artista, titulo, ano, pontos, peak, semanas):
    reg = {} #Cria o dicionario com a entrada atual
    #Adiciona os parametros ao dicionario
    reg['Artista'] = artista
    reg['Titulo'] = titulo
    reg['Ano'] = ano
    reg['Pontos'] = pontos
    reg['Peak'] = peak
    reg['Semanas'] = semanas
    
    #----------------------------------------------------------
    #Abrindo os arquivos
    with (open('database.bin', 'rb')) as openfile:
        database = pickle.load(openfile)
        
    with (open('indices.bin', 'rb')) as openfile:
        indices = pickle.load(openfile)
    
    with (open('topArtistas.bin', 'rb')) as openfile:
        topArtistas = pickle.load(openfile)
        
    with (open('topMusicas.bin', 'rb')) as openfile:
        topMusicas = pickle.load(openfile)
        
    #Abrindo os arquivos com as arvores Trie      
    with (open('trieMusicas.bin', 'rb')) as openfile:
        trieMusicas = pickle.load(openfile)
        
    with (open('trieArtistas.bin', 'rb')) as openfile:
        trieArtistas = pickle.load(openfile)
    #----------------------------------------------------------
    
    #inserindo no database
    database.append(reg)
    with (open('database.bin', 'wb+')) as openfile: #abre o arquivo no modo de leitura binaria 'wb' como openfile
        pickle.dump(database, openfile)
        
    #inserindo no arquivo de indices
    indNew = {}
    indNew['Ano'] = reg['Ano']
    indNew['Min'] = (len(database) - 1)
    indNew['Max'] = (len(database) - 1)
    
    indices.append(indNew)
    with (open('indices.bin', 'wb+')) as openfile: #abre o arquivo no modo de leitura binaria 'wb' como openfile
        pickle.dump(indices, openfile)
        
    #inserindo no topArtistas
    indArtista = -1 #flag que salva o indice do artista iterado, se for -1 o artista nao esta na lista
    indice = 0 #indice que esta procurando o artista
    for x in topArtistas: #percorre o arquivo topArtistas procurando pelo artista
        if x[0] == reg['Artista']:
            indArtista = indice
        indice += 1

    if(indArtista != -1): #Se o artista ja esta na lista
        topArtistas[indArtista] = (reg['Artista'], topArtistas[indArtista][1] + reg['Pontos'])
    else: #cria um novo artista
        topArtistas.append((reg['Artista'], reg['Pontos'])) #da um append com a tupla 'nome artista' + 'pontos'
        
    topArtistas.sort(key=lambda tup: tup[1], reverse=True)  #Faz um sort usando o elemento [1] (pontos) da tupla como criterio de ordenacao
    #Salva ranking geral de artistas
    with (open("topArtistas.bin", 'wb+')) as openfile:
        pickle.dump(topArtistas, openfile)
    
    #inserindo no topMusicas
    topMusicas.append((len(database)-1, reg['Pontos']))
    topMusicas.sort(key=lambda tup: tup[1], reverse=True)  #Faz um sort usando o elemento [1] (pontos) da tupla como criterio de ordenacao
    
    #Salva ranking geral de musicas
    with (open("topMusicas.bin", 'wb+')) as openfile:
        pickle.dump(topMusicas, openfile) 
      
    
    #inserindo nas arvores TRIE
    addString(trieMusicas, str(reg['Titulo']), len(database)-1)
    addString(trieArtistas, reg['Artista'], len(database)-1)
        
    filename = "trieMusicas.bin"
    with (open(filename, 'wb+')) as openfile: #abre o arquivo de teste no modo de leitura binaria 'wb' como openfile
        pickle.dump(trieMusicas, openfile)
    
    filename = "trieArtistas.bin"
    with (open(filename, 'wb+')) as openfile: #abre o arquivo de teste no modo de leitura binaria 'wb' como openfile
        pickle.dump(trieArtistas, openfile)